In [122]:
import pandas as pd
from sklearn.metrics import log_loss,roc_auc_score,roc_curve,accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [69]:
brupt = pd.read_csv("C:/Users\dai\Desktop\Machine Learning\Cases\Cases\Bankruptcy\Bankruptcy.csv")

In [70]:
le = LabelEncoder()
y = brupt['D']
X = brupt.drop(['NO','D'],axis=1)

In [98]:
glass = pd.read_csv("C:/Users\dai\Desktop\Machine Learning\Cases\Cases/Glass Identification/Glass.csv")
le = LabelEncoder()
y = le.fit_transform(glass['Type'])
X = glass.drop(['Type'],axis=1)
print(le.classes_)

lr = LogisticRegressio

['building_windows_float_processed' 'building_windows_non_float_processed'
 'containers' 'headlamps' 'tableware' 'vehicle_windows_float_processed']


In [99]:
lr=LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, 
                                                    random_state=24,
                                                   stratify=y)

In [111]:
scaler = StandardScaler().set_output(transform='pandas')
precomp = PCA().set_output(transform ='pandas')
X_trn_scl = scaler.fit_transform(X_train)
X_trn_pca = precomp.fit_transform(X_trn_scl)

print(np.cumsum(precomp.explained_variance_ratio_ * 100))
#principalComponents = pca.fit_transform(milk)


[ 27.79886944  50.63210658  66.37941685  78.83648726  88.60527685
  94.45669828  99.36141144  99.98176992 100.        ]


In [112]:
lr.fit(X_trn_pca, y_train)
#print(lr.coef_)
#print(lr.intercept_)


LogisticRegression()

In [113]:
X_tst_scl = scaler.transform(X_test)
X_tst_pca = precomp.transform(X_tst_scl)

In [114]:

y_pred = lr.predict(X_tst_pca)
y_pred_proba = lr.predict_proba(X_tst_pca)
print(accuracy_score(y_test,y_pred))
print(log_loss(y_test,y_pred_proba))

0.6153846153846154
0.9644514230861272


In [115]:
from sklearn.pipeline import Pipeline
pipe =Pipeline([('SCL',scaler),('PCA',precomp),('LR',lr)])
pipe.fit(X_train,y_train)

Pipeline(steps=[('SCL', StandardScaler()), ('PCA', PCA()),
                ('LR', LogisticRegression())])

In [116]:
y_pred = pipe.predict(X_test)
y_pred_proba = pipe.predict_proba(X_test)
print(accuracy_score(y_test,y_pred))
print(log_loss(y_test,y_pred_proba))

0.6153846153846154
0.9644514230861272


In [119]:
from sklearn.model_selection import GridSearchCV , StratifiedKFold

params={'PCA__n_components':[5,6,7,8,9],
        'LR__C':np.linspace(0.001,3,5),
        'LR__multi_class':['ovr','multinomial']
        }

kfold = StratifiedKFold(n_splits=5, shuffle=True, 
              random_state=24)
gcv = GridSearchCV(pipe, param_grid=params, cv =kfold ,scoring = 'neg_log_loss')
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__C': 0.75075, 'LR__multi_class': 'multinomial', 'PCA__n_components': 9}
-0.954645334455566


In [121]:
nb = GaussianNB()
pipe =Pipeline([('SCL',scaler),('PCA',precomp),('NB',nb)])
params={'PCA__n_components':[5,6,7,8,9]}
kfold = StratifiedKFold(n_splits=5, shuffle=True, 
              random_state=24)
gcv = GridSearchCV(pipe, param_grid=params, cv =kfold ,scoring = 'neg_log_loss')
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

C:\Users\dai\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
C:\Users\dai\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
C:\Users\dai\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
C:\Users\dai\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
C:\Users\dai\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


{'PCA__n_components': 5}
-1.7884651125063964


C:\Users\dai\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
C:\Users\dai\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


In [124]:
nb = RandomForestClassifier(random_state=24)
pipe =Pipeline([('SCL',scaler),('PCA',precomp),('RF',nb)])
params={'PCA__n_components':[5,6,7,8,9]}
kfold = StratifiedKFold(n_splits=5, shuffle=True, 
              random_state=24)
gcv = GridSearchCV(pipe, param_grid=params, cv =kfold ,scoring = 'neg_log_loss')
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

{'PCA__n_components': 8}
-0.7033348422541085
